In [54]:
import geopandas as gpd
from datetime import datetime
from libcomcat.search import search
import pandas as pd
import pprint

# Import

In [2]:

# =============================================================================
# =============================================================================

# --- 1. Paramètres utilisateur ---
geojson_path = "myanmar.json"    # <- à adapter
date_start = "2025-03-22"   # au format YYYY-MM-DD
date_end   = "2025-04-30"
# =============================================================================
# =============================================================================



# --- 2. Charger le GeoJSON et s'assurer du bon CRS ---
gdf = gpd.read_file(geojson_path)
if gdf.crs is not None and gdf.crs.to_epsg() != 4326:
    gdf = gdf.to_crs(4326)
minx, miny, maxx, maxy = gdf.total_bounds

# --- 3. Conversion des dates ---
starttime = datetime.fromisoformat(date_start)
endtime = datetime.fromisoformat(date_end)

# --- 4. Recherche des événements ---
events = search(
    minlongitude=minx,
    minlatitude=miny,
    maxlongitude=maxx,
    maxlatitude=maxy,
    starttime=starttime,
    endtime=endtime,
)

# --- 5. Afficher les attributs du premier événement pour inspecter ---
if events:
    first = events[0]
    print(first)
    print("----------- Attributs disponibles ----------")
    print(dir(first))
else:
    print("Aucun événement trouvé.")

# --- 6. Construire le DataFrame selon les attributs disponibles ---
# Liste possible des attributs à récupérer
cols = ["id", "time", "magnitude", "depth", "latitude", "longitude", "magtype", "gap", "rms", "horizontal_error", "vertical_error"]

data = []
for e in events:
    d = {}
    for c in cols:
        d[c] = getattr(e, c, None)
    data.append(d)
df = pd.DataFrame(data)
print(df.head())

# Tu pourras adapter la liste 'cols' en fonction de ce que tu vois dans le dir(first)


us6000q3p6 2025-03-24 04:38:32.700000 (29.189,93.747) 10.0 km M4.2
----------- Attributs disponibles ----------
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_jdict', 'alert', 'depth', 'getDetailEvent', 'getDetailURL', 'hasProduct', 'hasProperty', 'id', 'latitude', 'location', 'longitude', 'magnitude', 'properties', 'time', 'toDict', 'url']
           id                             time  magnitude  depth  latitude  \
0  us6000q3p6 2025-03-24 04:38:32.700000+00:00        4.2   10.0   29.1888   
1  us6000q3ny 2025-03-24 04:43:00.743000+00:00        4.1   10.0   29.2119   
2  us7000pm7w 2025-03-24 11:21:38.311000+00:00        5.0   10.0   28.1045   
3  us7000pmg7 

In [3]:
df = df.sort_values("magnitude", ascending=False)
df

,id,time,magnitude,depth,latitude,longitude,magtype,gap,rms,horizontal_error,vertical_error
5,us7000pn9s,2025-03-28 06:20:52.715000+00:00,7.7,10.0,22.0110,95.9363,None,None,None,None,None
6,us7000pn9z,2025-03-28 06:32:04.777000+00:00,6.7,10.0,21.6975,95.9690,None,None,None,None,None
66,us6000q5ps,2025-04-13 02:24:57.717000+00:00,5.3,10.0,21.2292,96.0897,None,None,None,None,None
78,us7000pv6f,2025-04-26 04:21:49.088000+00:00,5.1,10.0,31.0900,98.9820,None,None,None,None,None
30,us7000pnkj,2025-03-29 09:20:48.229000+00:00,5.1,10.0,19.6858,96.0761,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
64,us6000q5cs,2025-04-11 10:52:56.066000+00:00,4.1,10.0,24.0430,91.3615,None,None,None,None,None
59,us6000q575,2025-04-10 16:17:58.886000+00:00,3.9,10.0,21.7193,95.9475,None,None,None,None,None
56,us6000q4x4,2025-04-09 14:40:17.442000+00:00,3.6,10.0,21.4447,95.9352,None,None,None,None,None
57,us6000q4xc,2025-04-09 15:59:18.014000+00:00,3.6,10.0,21.9457,96.0058,None,None,None,None,None


## Création d'un geo df

In [4]:

import geopandas as gpd
from shapely.geometry import Point

gdf_map = gpd.GeoDataFrame(
    df,
    geometry=[Point(xy) for xy in zip(df.longitude, df.latitude)],
    crs="EPSG:4326"
)


## Check sur une carte

In [5]:
# Load a map with data and config and height
from keplergl import KeplerGl
map_1 = KeplerGl(height=1000, show_docs=False)
map_1.add_data(data=df, name="Séismes")
map_1


# map_1.save_to_html(file_name="data_output/seismes_map.html")


KeplerGl(data={'Séismes': {'index': [5, 6, 66, 78, 30, 2, 77, 48, 8, 9, 7, 44, 61, 31, 43, 76, 68, 81, 47, 35,…

In [6]:
import folium
import numpy as np  # pour np.exp

# Exemple : Rayon exponentiel en fonction de la magnitude
def mag_to_radius(mag):
    # Version exponentielle "douce" :
    return 2 * np.exp(mag - 4)  # ajuster le facteur si besoin


# Création de la carte
m = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()], zoom_start=6, tiles="cartodbpositron")

for _, row in df.iterrows():
    radius = mag_to_radius(row["magnitude"]) if row["magnitude"] else 2
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=radius,
        popup=f"Mag {row['magnitude']}<br>ID: {row['id']}",
        color="#3186cc",
        fill=True,
        fill_opacity=0.6
    ).add_to(m)

m  # affichage dans le notebook


In [38]:
event_id = df.iloc[0]["id"]
print(event_id)  # Ex: 'us6000kikc'


us7000pn9s


### Accès à toutes les données de Shakemap

In [55]:
import requests
import json

event_id = "us7000pn9s"
url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?eventid={event_id}&format=geojson"
r = requests.get(url)
data = r.json()
data

{'type': 'Feature',
 'properties': {'mag': 7.7,
  'place': '2025 Mandalay, Burma (Myanmar) Earthquake',
  'time': 1743142852715,
  'updated': 1751002490930,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us7000pn9s',
  'felt': 2526,
  'cdi': 9.1,
  'mmi': 9.953,
  'alert': 'red',
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 2910,
  'net': 'us',
  'code': '7000pn9s',
  'ids': ',us7000pn9s,usauto7000pn9s,pt25087002,at00sttln4,',
  'sources': ',us,usauto,pt,at,',
  'types': ',dyfi,earthquake-name,finite-fault,general-text,ground-failure,impact-text,internal-moment-tensor,internal-origin,losspager,moment-tensor,oaf,origin,phase-data,shakemap,',
  'nst': 246,
  'dmin': 3.474,
  'rms': 0.89,
  'gap': 24,
  'magType': 'mww',
  'type': 'earthquake',
  'title': 'M 7.7 - 2025 Mandalay, Burma (Myanmar) Earthquake',
  'products': {'dyfi': [{'indexid': '26627303',
     'indexTime': 1751002496395,
     'id': 'urn:usgs-product:us:dyfi:us7000pn9s:1751002490930',
     'ty

In [53]:
from datetime import datetime

files = []

for product_type, product_list in products.items():
    for product in product_list:
        contents = product.get('contents', {})
        if isinstance(contents, dict):
            for file_name, file_info in contents.items():
                # Récupérer le timestamp s'il existe
                lastmod = file_info.get("lastModified")
                # Conversion si présent
                if lastmod is not None:
                    # lastmod est en millisecondes
                    lastmod_str = datetime.utcfromtimestamp(int(lastmod) / 1000).strftime('%Y-%m-%d %H:%M:%S')
                else:
                    lastmod_str = "unknown"
                files.append({
                    "product_type": product_type,
                    "file_name": file_name,
                    "url": file_info.get("url"),
                    "size": file_info.get("length", "unknown"),
                    "content_type": file_info.get("contentType", "unknown"),
                    "last_modified": lastmod_str
                })

files = sorted(files, key=lambda x: (x['product_type'], x['file_name']))

for f in files:
    print(
        f"{f['product_type']:16} {f['file_name']:40} "
        f"{f['size']:>10} bytes  {f['last_modified']:19}  {f['url']}"
    )



dyfi             cdi_geo.txt                                   33462 bytes  2025-06-27 05:17:03  https://earthquake.usgs.gov/product/dyfi/us7000pn9s/us/1751001424084/cdi_geo.txt
dyfi             cdi_geo.xml                                  110595 bytes  2025-06-27 05:17:03  https://earthquake.usgs.gov/product/dyfi/us7000pn9s/us/1751001424084/cdi_geo.xml
dyfi             cdi_geo_1km.txt                              115151 bytes  2025-06-27 05:17:03  https://earthquake.usgs.gov/product/dyfi/us7000pn9s/us/1751001424084/cdi_geo_1km.txt
dyfi             cdi_zip.txt                                   23085 bytes  2025-06-27 05:17:03  https://earthquake.usgs.gov/product/dyfi/us7000pn9s/us/1751001424084/cdi_zip.txt
dyfi             cdi_zip.xml                                   48598 bytes  2025-06-27 05:17:03  https://earthquake.usgs.gov/product/dyfi/us7000pn9s/us/1751001424084/cdi_zip.xml
dyfi             contents.xml                                   4224 bytes  2025-06-27 05:17:03  https://e

/var/folders/8b/jbf_9gw94xs45f1kt0j8k5880000gq/T/ipykernel_32708/2887742443.py:15: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  lastmod_str = datetime.utcfromtimestamp(int(lastmod) / 1000).strftime('%Y-%m-%d %H:%M:%S')
